# Creation of Signal Detection Data (1/3):
This script is meant to be run after the accompaning mysql script that creates the requisite tables, and before the other two python scripts. It uses signal detection theory to create the features dprime, beta, and C for each individual based off their omission/commission errors.

Note that some fields and tables will need to be changed amongst all of these as we implement the final database. IE snap_hack will be in the psych profile, adhd feature will be in the SIQ, etc.

### 1. Necessary Imports:

In [1]:
import numpy as np
import math 
from scipy import stats
import pandas as pd
import pymysql


from __future__ import division # Not neccessary in Python 3 and later
import scipy
from math import exp,sqrt

### 2. Functions to estimate D', Beta, and C

In [2]:
Z = stats.norm.ppf

#This function calculates D', Beta, and C
def dPrime(hits, misses, fas, crs):
    # Floors an ceilings are replaced by half hits and half FA's
    halfHit = 0.5/(hits+misses)
    halfFa = 0.5/(fas+crs)
 
    # Calculate hitrate and avoid d' infinity
    hitRate = hits/(hits+misses)
    if hitRate == 1: hitRate = 1-halfHit
    if hitRate == 0: hitRate = halfHit
 
    # Calculate false alarm rate and avoid d' infinity
    faRate = fas/(fas+crs)
    if faRate == 1: faRate = 1-halfFa
    if faRate == 0: faRate = halfFa
 
    # Return d', beta, c and Ad'
    out = {}
    dPrizime = Z(hitRate) - Z(faRate)
    beta = exp((Z(faRate)**2 - Z(hitRate)**2)/2)
    c = -(Z(hitRate) + Z(faRate))/2
    Ad = stats.norm.cdf(dPrizime/sqrt(2))
    return dPrizime, beta, c

In [3]:
#This function processes ommission and commission errors, 
#and send them out to estimate d', beta, C function
def generate_signal_detection(omissionErrors, commissionErrors, trueSigs, falseSigs):
    
    hits = trueSigs - omissionErrors
    misses = omissionErrors
    falseAlarms = commissionErrors
    correctRejections = falseSigs - commissionErrors
    
    dP = np.zeros(20)
    beta = np.zeros(20)
    C = np.zeros(20)

    for i in np.arange(np.shape(omissionErrors)[0]):
        dP[i], beta[i], C[i] = dPrime(hits[i], misses[i], falseAlarms[i], correctRejections[i])
    return np.column_stack((dP, beta, C))   

In [4]:
#This funtion slices up raw table into requisite chunks

def separate_blocks(data):
    #Creating adhd vector
    adhd = np.zeros((20, 1))
    adhd[:, 0] = np.array((1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1))

    #creating blocks:
    block1 = data[data[:, 0] ==1, :]
    block1 = block_to_signal(block1, adhd, 17, 179) #not accurate, need to double check
    
    block2 = data[data[:, 0] ==2, :]
    block2 = block_to_signal(block2, adhd, 17, 179) #not accurate, need to double check
    
    block3 = data[data[:, 0] ==3, :]
    block3 = block_to_signal(block3, adhd, 18, 180) #not accurate, need to double check

    total = data[data[:, 0] ==0, :]
    total =  block_to_signal(total, adhd, 52, 538 - 52) #not accurate, need to double check
    
    #merging blocks into final table:
    finalData = np.array((block1[0, :], block2[0, :], block3[0, :], total[0, :]))

    for i in np.arange(1, np.shape(total)[0]):
        holder = np.array((block1[i, :], block2[i, :], block3[i, :], total[i, :]))
        finalData = np.concatenate((finalData, holder))
    
    return finalData

In [5]:
#This function serves as an intermediary between separate-blocks and generate_signal_detection
def block_to_signal(block, adhd, trueSigs, falseSigs):
    block = np.concatenate((block, adhd), axis = 1)
    blockSignal = generate_signal_detection(block[:, 2], block[:, 3], trueSigs, falseSigs)
    block = np.concatenate((block, blockSignal), axis = 1)
    return block

### 3. DB connecting, pull, & insert

In [6]:
#initializes connection info for database
def connect():
    return pymysql.connect(host = "rm-j6cluj4576jdi6n6oo.mysql.rds.aliyuncs.com",
                           database = 'rnd_test', 
                           user='cognitiveleap', 
                           password= 'QWE@123456')

In [7]:
#Gets CPT data from database
def get_data(CaseIds, raw):
    #connect to db
    db = connect()
    
    # prepare a cursor object using cursor() method
    cursor = db.cursor()
    data = []

    # Prepare SQL query to INSERT a record into the database.
    for i in CaseIds:
        if raw:
            sql = """SELECT Block, TargetsRtVariability, OmissionErrors, CommissionErrors, CasdId 
                     FROM cpt_output_results WHERE CasdId = """ + str(i) 
            cursor.execute(sql)
            # Fetch all the rows in a list of lists.
            results = np.asarray(cursor.fetchall())
            data.append(results)
        else:
            sql = "SELECT PathLen, TimeActive, NumRot, TotalDeg, CasdId FROM head_features WHERE CasdId = %s "% str(i)
            #print(sql)
            cursor.execute(sql)
            results = np.asarray(cursor.fetchall())
            if (len(results) > 0):
                data.append(results[0])
     
        #odd error
        #except:
         #print ("Error: unable to fetch data")"""
    db.close()
    
    return np.concatenate(np.asarray(data))

In [8]:
#Inserts created features into signal detection tables

def insert_signal(CaseIds, data):
    db = connect()
    cursor = db.cursor()
    for i in range(len(data)):
        sql = "INSERT INTO signal_detection"
        sql += "(Id, CaseID, Block, ADHD, DPrime, Beta, C)"
        sql += "VALUES (%s, %s, %s, %s, %s, %s, %s)" % (np.int(i),
                                                                    np.int(data[i, 4]),                                                                       
                                                                    np.int(data[i, 0]),                                                                       
                                                                    np.bool(data[i, 5]),
                                                                    data[i, 6], 
                                                                    data[i, 7], 
                                                                    data[i, 8])
        #sql = "SHOW COLUMNS FROM "
        # Execute the SQL command
        cursor.execute(sql)
        # Commit your changes in the database
        db.commit()
    db.close()

In [9]:
#this function inserts the manually entered SNAP data from the original 20 taiwan participants. 
#We hacked it together for testing. 

def insert_snap(caseIds):
        #Adding copied and paseted SNAP values:
    snapINN = np.zeros((20, 1))
    snapINN[:, 0] = np.array((19, 9, 12, 1, 0, 19, 3, 1, 3, 12, 5, 5, 4, 5, 13, 16, 17, 12, 8, 3))
    snapIMP = np.zeros((20, 1))
    snapIMP[:, 0] = np.array((16, 9, 4, 0, 0, 11, 1, 1, 0, 6, 6, 6, 5, 1, 3, 16, 12, 9, 14, 4))
    #snapSUM = snapIMP[:, 0] + snapINN[:, 0]
    
    db = connect()
    cursor = db.cursor()
    for i in range(20):
        sql = "INSERT INTO snap_hack"
        sql += "(CaseID, SNAPINN, SNAPIMP, SNAPCombined)"
        sql += "VALUES (%s, %s, %s, %s)" % (caseIds[i],
                                            snapINN[i, 0],
                                            snapIMP[i, 0],
                                            snapINN[i, 0] + snapIMP[i, 0])
        #sql = "SHOW COLUMNS FROM "
        # Execute the SQL command
        cursor.execute(sql)
        # Commit your changes in the database
        db.commit()
    db.close()

### 4. Main Function:

In [10]:
#main function
def main_bayes(caseIds):
    allPatients = get_data(caseIds, True)
    allPatients = separate_blocks(allPatients)
    insert_signal(caseIds, allPatients)
    insert_snap(caseIds)

    

In [11]:
#calls main function, stores data for observation
allPatients = main_bayes(np.arange(1, 21, 1))
